# HD BITCOIN WALLET

This project aims to implement and Hierarchical Deterministic(HD) wallet using the bitcoin standards BIP-32 
and BIP-39 by following the guidelines from the book Mastering Bitcoin. An HD wallet allows to derive many keys 
from a single seed stored in a tree structure. With respect to non deterministic wallets, where keys are generated 
at random and have no connection with one another, the HD wallet allows for simpler managment and doesn't require
to keep copies of all the keys since they can all be derived from the seed. Furthermore HD wallets allow to derive 
public keys without the need to access to the associated private keys which guarantees additional security in 
insecure scenarios.

<img src="pic.png",width="500", height= "300">

In [1]:
"""Now we import all the required libraries"""

from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure, every node has a parent, a private key,
a chain code and a list of children"""

class TreeNode():
    
    def __init__(self,parent,pvt_key,pub_key,chain_code): #every node has a parent, the master will have None as input
        self.parent=parent
        self.pvt_key=pvt_key 
        self.pub_key=pub_key
        self.chain_code =chain_code
        self.children = []
    

Now we need to follow the seed generation procedure as described by BIP-39. The whole process is performed in a few commands thanks to the Mnemonic library.The concept is of generating a random entropy which is then mapped to some words (mnemonic words), which can be used to generate the actual seed.The idea of using mnemonic words comes from the fact that they are much simpler to remember than the seed itself and can be used to unambigously retrive all of the keys in the wallet, allowing for a backup in case of loss or destruction. The whole procedure is explained in the following images
<img src="pic2.png" style ="width:700px; height: 500px">
<img src="pic3.png" style ="width:700px; height: 500px">


In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght): #strenght refers to the lenght in bits of the entropy to generate 
                                                 #passphrase is an additional value used to append to the salt for PBKDF2
    
   mnemo = Mnemonic(language)

   words = mnemo.generate(strength=strenght) #this command performs steps 1 to 5 and saves the words in a vavariable
    
   seed = mnemo.to_seed(words, passphrase=passphrase)#here we retrieve the seed from the words (step 7 to 9)

   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
    
   print("Words= " + words)   

   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
    
   return seed 
   


In [4]:
#Here we simply test our function and check the values generated 
seed=generate_seed("english","test",128)

Entropy=b'f81d26f3ec75d155a87c33a847b6b95a'
Words= way truck rude suggest fringe print peanut major portion diet purity rely
Seed = b'47e0a60ed22983229295a5d118f0ed7e16a36e62e6833ef2f89955f5b4f8b20c86157052b8d0d3806cbfa661d7f0c4a232410f967161e65e3a75c2a731c0335c'


Now that we have the seed it's time to generate the master. We make use of HMAC-SHA512 in order to have a fixed lenght output and to protect the seed from some possible attacks(to this end the hash is one way).It is worth noticing that the public key is then generated starting from the private key using ellipric curve theory, where public key = G*private key and G is the generator point of the curve. As the theory suggests, the public key is a pair of coordinates x and y, so to obtain the actual public key I take the first two bytes of the y coordinate and combine them with all the ones from the x axis. The following picture explains the process:
<img src="pic4.png" style ="width:600px; height: 400px">

In [5]:
"""In this section I derive from the seed the master private key and the master chain code, 
the private key is the one used to have control over your bank account represented by the public key. 
The bitcoin address is generated/corresponds from/to the public key"""

def get_master_info(seed):
    
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest() #here we perform HMAC-SHA512 on the root seed
    
    master_pvt_key=master_hmac[:64] #we pick the first 256 bits (256/4=64 since we are working in hex) 
    
    master_chain_code=master_hmac[64:] #we extract the chain code using a similar logic 
    
    #Now we need to derive the public key for our master and I have used fastecdsa for a quick result
    
    pvt=int(master_pvt_key, 16) #some conversion for compliance with the function 
    
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1) # we generate the public key using the secp256k1 elliptic curve 
    
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #for the serialization of the public key i used the hints at: https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    
    master=TreeNode(None,master_pvt_key,pub_key,master_chain_code) #I create a TreeNode instance and pass as parent None, since we are creating the root 
    
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pvt_key


'5ee4ed711f43c3686ff6967760e9754d102062e5e85c676c8e9731325aa1d09d'

Now that we have the master we have to create the children.In order to do so we need the parent's public or private key, the child index and the parent chain code. The last one is needed in order to introduce deterministic random data so that something more is required than the index and the child key to get to the siblings of a determined node. 
We have two types of children:
1) regular children which are derived using the public address of the parent
<img src="pic6.png" style ="width:600px; height: 400px">
2) hardened children which are derived from the private address of the parent
<img src="pic5.png" style ="width:600px; height: 400px">
Hardened children are particularly useful when we are with public extended keys where we need to deal with the possibility of having a children private key leaked. Since from the extended public address we can derive the chain code it is possible using the two to get the private keys of other children or even worse the one of the master. So we want to break the relationship between parent public key and child chain code.In this way the public key doesn't give any useful info on the other children private keys.In this way we can create extended public keys which are not vulnerable.


In [7]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex, num represents the number od children we want to create 
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 

In [8]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:]) #here I concatenate all the values that should be hashed 
        
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest() #Here I perform HMAC-SHA512 on the input string
        
        pvt_key=child_info[:64] #similarly as the master we extract the private key
        
        chain_code=child_info[64:]
        
        pvt=int(pvt_key, 16) #compliance transformations 
        
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #same as master 
        
        child=TreeNode(parent.pvt_key,pvt_key,pub_key,chain_code) #I create the TreeNode entity
        
        parent.children.append(child) #I append the new entity to the list of children of the parent 
            

Now we are ready to create some children, as an example I create two regular children and and one hardened children. 

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")

Private Key 95efd4ae85063103d568471f5de17ddf9ec14d1379b07d789cf5a2640c8b896a
Public Key 5351d695a55a57dd858124f89e32eb927059107e0006f05934b24c891f0144818c
Chain Code f1b752fc7ca15aa1b0163498ac3d9bd96a122ddedec0d3213a543c79cb09c369
-------------------------------------------------------------------------
Private Key 0c964c1b05c9f2117f721f82b764ee749dd4e0086e8186055cd57cf5c7797e2b
Public Key f7831166421c7468c5bbcb2ca077eb854072c9fb1bc742b9c4b4bbba54c835c17d
Chain Code 6ec25ea3bd44d694685d0a78b3cf58abc5154ec6ba2b7eb84112b1b3d59a8c75
-------------------------------------------------------------------------
Private Key df80097b7cf9ece0469ea7f921238bdae2453ef056c94720042a7bac79620dac
Public Key 96681806ec980540f83bcbfa5e6a7e0fe4bfcff26bde63d34415e9e97b3c4ceab6
Chain Code 2ba99b09154528efd1968cc276146e9fe788e6f03d04b4713ba70f3b172e02c1
-------------------------------------------------------------------------


Now that we have some children let's create some grandkids. Each child will have one hardened children and two regular children. The result will show a total of nine nodes (3x3=9)

In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 7b5d688ab8c6f1c8ee3248ebedf9a7dfe5f1e4dea6fff2fe33b93ee95e9d025d
Public Key 6ff1e79767b239d11290bd7206458d054280f6227ecaf454207c2884a59af81e85
Chain Code efe85c2a2d5387b7c3a9b146bbc4bfd3f4a77db5ada7a580c2b0d9f1090feee7
-------------------------------------------------------------------------
Private Key ead8441ba7d24367899d92064b5e430f8c669274cb9c4a2a3458bdb833e9dda1
Public Key 14690d9da04726a44a0dc1fa66ad2196f761256867eab7adc8eb8471bebf104096
Chain Code 600a4ca580a58478182ce26163e078633c2a30c584a9ddcc796cfa472c61fbd9
-------------------------------------------------------------------------
Private Key ac66fe9b9c7f128fe0787496c91634682070108ef54d4fc615bcdc8f386fcd0e
Public Key e6d8b41d36a5325bd1ed6f59c0553dd29df86b04a8ab2440a236f60d38ddd4f5a8
Chain Code cbdcfdd02accb53981c7029f8a7990bde7465afbdd1930cca5ca431adbca4a22
-------------------------------------------------------------------------
Private Key 30937162c8e2eaf9f1892bba58dbea63308c46ab364904192f5b00510ad1eec0
Public K